In [1]:
import pandas as pd

In [16]:
ref_table = pd.read_parquet('game_reference_table.parquet')

In [17]:
ref_table = ref_table.set_index('name')

In [52]:
def get_genres(game_name: str) -> str:
    try:
        game_name_processed = game_name.lower().replace(' ', '-')
    except:
        raise ValueError('Text processing failed. Are you sure you used a string?')
    game_genres = ref_table.loc[game_name_processed]['genre_name']

    return list(game_genres)
    


In [47]:
def get_genre_recs(game_name: str, genre: str) -> str:
    tag_df = pd.read_parquet('./tag_rec_tables/' + str(genre) + '_rec_table.parquet')
    game_recs = tag_df.loc[game_name]
    order = game_recs.sort_values(ascending=False)
    return order.head(n=20)

In [82]:
def search_and_sort(game_name: str) -> str:
    game_rec_series = pd.Series()
    genres = get_genres(game_name)
    for genre in genres:
        recs = get_genre_recs(game_name=game_name, genre=genre)
        game_rec_series = game_rec_series.append(recs)
    game_rec_series
    game_rec_combo = game_rec_series.groupby(level=0, axis=0).sum().sort_values(ascending=False)
    return game_rec_combo.head(5)